<a href="https://colab.research.google.com/github/balandongiv/apid/blob/main/openai_whisper_python_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI Whisper and Auto Capture Frame

## 1. Getting Started with OpenAI Whisper

### a. Install OpenAI Whisper Python Library

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -U openai-whisper
!pip install pytube -q
!pip install pydub
!pip install python-docx

In [ ]:
from pydub import AudioSegment
import os
from pytube import YouTube
import re
import pandas as pd
from pytube import YouTube
from moviepy.editor import VideoFileClip
from PIL import Image
import os
from docx import Document
from docx.shared import Inches
from PIL import Image
import whisper
model = whisper.load_model("small")

In [ ]:
def clean_and_format_string(input_string):
    # Remove special characters and spaces, and replace them with underscores
    cleaned_string = re.sub(r'[^a-zA-Z0-9]+', '_', input_string)

    # Remove leading and trailing underscores
    cleaned_string = cleaned_string.strip('_')

    # Convert to lowercase if needed
    formatted_string = cleaned_string.lower()

    return formatted_string

### b. Load the OpenAI Whisper Model

In [ ]:
# from pydub import AudioSegment
# import os

def segment_audio(audio_path, segment_duration_seconds=20):  # segment_duration in seconds
    # Convert segment duration to milliseconds
    segment_duration_milliseconds = segment_duration_seconds * 1000

    # Load audio file
    audio = AudioSegment.from_file(audio_path)

    # Create a directory to store segments
    base_path, ext = os.path.splitext(audio_path)
    segment_dir = base_path + "_segments"
    if not os.path.exists(segment_dir):
        os.makedirs(segment_dir)

    # Segment audio and export
    segment_paths = []
    for i in range(0, len(audio), segment_duration_milliseconds):
        segment = audio[i:i + segment_duration_milliseconds]
        segment_path = os.path.join(segment_dir, f"segment_{i // segment_duration_milliseconds}{ext}")
        segment.export(segment_path, format=ext.replace('.', ''))
        segment_paths.append(segment_path)

    return segment_paths


In [ ]:
def download_audio_from_youtube(youtube_video_url):
    # folder_storage = 'Concept Selection process'
    youtube_video_content = YouTube(youtube_video_url)
    video_name = youtube_video_content.streams[0].default_filename

    folder_storage = clean_and_format_string(video_name)


    audio_streams = youtube_video_content.streams.filter(only_audio=True)
    # for stream in audio_streams:
    #     print(stream)

    audio_stream = audio_streams[1]
    dpath = audio_stream.download(folder_storage)
    # print(audio_stream)
    return dpath

Get png image segment

In [ ]:
def download_youtube_video(url, path='video.mp4'):
    """Download a YouTube video using pytube and return the title."""
    print("Downloading YouTube video...")
    yt = YouTube(url)
    video_title = yt.title
    output_fname = create_valid_folder_name(video_title)
    path=f'{output_fname[:20]}.mp4'
    stream = yt.streams.filter(progressive=True, file_extension='mp4').first()
    stream.download(filename=path)
    print(f"Video {video_title} has been downloadeded successfully.")
    return path, video_title

def create_valid_folder_name(title):
    """Create a valid folder name from the video title."""
    valid_name = ''.join(char for char in title if char.isalnum() or char in [' ', '_']).rstrip()
    return valid_name.replace(' ', '_')

def format_frame_name(seconds):
    """Format the frame name using minute and second format."""
    minutes = seconds // 60
    seconds = seconds % 60
    # return f"frame_{minutes:02d}_{seconds:02d}.png"
    return f"{minutes:02d}_{seconds:02d}.png"

def capture_frames(video_path, interval=5, output_folder='frames'):
    """Capture frames from the video at given interval."""
    print(f"Capturing frames from the video into folder '{output_folder}'...")
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    all_path=[]
    with VideoFileClip(video_path) as video:
        duration = int(video.duration)
        for i in range(0, duration, interval):
            frame = video.get_frame(i)
            frame_image = Image.fromarray(frame)
            frame_name = format_frame_name(i)
            frame_path = os.path.join(output_folder, frame_name)
            # print(f'what is frame path {frame_path}')
            frame_image.save(frame_path)
            # print(f"Captured and saved {frame_name}.")
            all_path.append(frame_path)
    print("All frames have been captured and saved.")
    return all_path

In [ ]:


def calculate_image_dimensions(image_path, max_width, max_height):
    # Initialize variables to store the new width and height
    new_width = None
    new_height = None

    # Check the first image to calculate the new dimensions
    first_image = Image.open(image_path)
    first_image_width, first_image_height = first_image.size

    # Calculate the scaling factor based on the first image
    width_ratio = max_width / first_image_width
    height_ratio = max_height / first_image_height
    scale_factor = min(width_ratio, height_ratio)

    # Calculate the new width and height based on the first image
    new_width = first_image_width * scale_factor
    new_height = first_image_height * scale_factor

    return new_width, new_height

def add_images_to_word_document(df, word_fname, max_width, max_height, num_images=10):
    print('Now I am saving the all the transcription and images in word')


    #     # Check if word_fname already exists
    # if os.path.isfile(word_fname):
    #     # If it exists, create a new name with an addition (e.g., v1, v2, v3)
    #     base_name, extension = os.path.splitext(word_fname)
    #     counter = 1
    #     while os.path.isfile(word_fname):
    #         word_fname = f'{base_name}_v{counter}{extension}'
    #         counter += 1


            # Check the first image in the loop to calculate the new dimensions
    first_image_path = df.iloc[0]['image_path']




    new_width, new_height = calculate_image_dimensions(first_image_path, max_width, max_height)
    doc = Document()
    for index, row in df.iterrows():
    # for index, row in df.head(num_images).iterrows():
        # Add the image
        image_path = row['image_path']

        doc.add_picture(image_path, width=Inches(new_width), height=Inches(new_height))
        # transcription_text = row['Transcription']['text']

        doc.add_paragraph(row['Transcription']['text'])

        # Add a new line between transcriptions
        doc.add_paragraph("\n")

    # Save the document
    doc.save(word_fname)
    print('Complete saving the all the transcription and images in word')
    print(f'The file is saved as  {word_fname}')

In [ ]:
def sanitize_filename(title):
    # Define characters that should be removed or replaced
    remove_chars = ['\\', '/', ':', '*', '?', '"', '<', '>', '|']
    replace_with_underscore = [' ', '-']

    # Replace or remove characters
    for char in remove_chars:
        title = title.replace(char, '')
    for char in replace_with_underscore:
        title = title.replace(char, '_')

    # Optionally, you can also limit the length of the file name
    max_length = 255  # Maximum file name length for most file systems
    return title[:max_length]

In [ ]:
def process_video_now(video_title,video_path,durat,output_word_file):
    print('step 1')
    output_folder = create_valid_folder_name(video_title)

    # downloaded_audio_path = download_audio_from_youtube(youtube_video_url)
    downloaded_audio_path=video_path
    segments = segment_audio(downloaded_audio_path, segment_duration_seconds=durat)


    print('step 2')
    all_path = capture_frames(video_path, interval=durat, output_folder=output_folder)

    df2 = pd.DataFrame(all_path, columns=['image_path'])

    df = pd.DataFrame(segments, columns=['SegmentPath'])

    print('step 3')
    df['SegmentNumber'] = df['SegmentPath'].apply(lambda x: int(x.split('_')[-1].split('.')[0]))
    df = df.sort_values(by='SegmentNumber')

    print(' This may take some time, but i am current whispering shhhhhh')
    df['Transcription'] = df['SegmentPath'].apply(lambda x: model.transcribe(x))
    combined_df = pd.concat([df2, df], axis=1)
    # Set the maximum width and height for the image (in inches)
    max_width = 6.5  # 8.5 inches - 1 inch margin on each side
    max_height = 9.0  # 11 inches - 1 inch margin on top and bottom


    add_images_to_word_document(combined_df, output_word_file, max_width, max_height, num_images=10)

In [ ]:
def process_video(durat, youtube_video_url):
    # Rest of your existing code

    video_url = youtube_video_url
    video_path, video_title = download_youtube_video(video_url)

    clean_fname=sanitize_filename(video_title)
    output_word_file=f'{clean_fname}.docx'

    # Directory to save files if Google Drive is not available
    local_save_directory = '/content/'
    gdrive='/content/drive/MyDrive/'
    # Check if Google Drive is mounted, if not, use the local directory
    save_directory = gdrive if os.path.exists(gdrive) else local_save_directory
    word_fname=save_directory+output_word_file

    if not os.path.exists(word_fname):
      # Do all the function
      c=1
      print('step 1b')
      process_video_now(video_title,video_path,durat,word_fname)
    else:
        print(f"Document '{word_fname}' already exists. Skipping.")


In [ ]:
# youtube_url="https://youtu.be/LWscm2qfnNQ?list=PLn9ADapr5YGhG0lcPnrObqOFFsz3hnj1t"
# duration=20

# process_video(duration, youtube_url)

Download from playlist

In [ ]:
from pytube import Playlist, YouTube

def get_playlist_urls(playlist_url):
    # Create a playlist object
    playlist = Playlist(playlist_url)

    # List to store video URLs
    video_urls = []

    # Iterate through each video in the playlist
    for url in playlist.video_urls:
        print(f'Found video URL: {url}')
        video_urls.append(url)

    return video_urls

# Replace with your playlist URL
playlist_url = 'https://www.youtube.com/playlist?list=PLOzRYVm0a65epMo6DDrlT0X5ejGWDbT13'
video_urls_playlist = get_playlist_urls(playlist_url)

# Optionally, print all URLs


In [ ]:
# for url in video_urls_playlist:
#     print(url)

In [ ]:
def get_youtube_video_title(url):
    try:
        # Create a YouTube object
        yt = YouTube(url)
        # Get the title of the video
        title = yt.title
        return title
    except Exception as e:
        return f"An error occurred: {e}"

# Example URL
# url = "https://www.youtube.com/watch?v=G-JN2vFZPvI"

# # Get the video title
# print(get_youtube_video_title(url))

In [ ]:
import docx
def create_docx_for_each_title(titles,err_or):
    sanitized_title = sanitize_filename(titles)
    # Directory to save files if Google Drive is not available
    local_save_directory = '/content/'
    gdrive='/content/drive/MyDrive/'
    # Check if Google Drive is mounted, if not, use the local directory
    save_directory = gdrive if os.path.exists(gdrive) else local_save_directory
    filename=save_directory+f'{sanitized_title}.docx'



    # Check if file already exists
    if not os.path.exists(filename):
          doc = docx.Document()
          doc.add_paragraph(titles)
          # doc.add_paragraph(err_or)
          doc.save(filename)
          print(f"Document '{filename}' created.")
    else:
        print(f"Document '{filename}' already exists. Skipping.")

In [ ]:
duration = 20
for youtube_urlx in video_urls_playlist:
    try:
        process_video(duration, youtube_urlx)
    except Exception as e:
        ytitle=get_youtube_video_title(youtube_urlx)
        create_docx_for_each_title(ytitle,e)
        print(f"An error occurred while processing {youtube_urlx}: {e}")
